


# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Problem Statement

The problem is to identify the author of a  book from a given list of possible authors.

## Learning Objectives

At the end of the experiment, you will be able to:

* Use NLTK package
* Extract handcrafted features 
* Preprocess the text
* Write an algorithm to identify author of a given book


In [ ]:
#@title  Mini Hackathon Walkthrough
from IPython.display import HTML

HTML("""<video width="320" height="240" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Walkthrough/authoridentification.mp4" type="video/mp4">
</video>
""")

## Background 

Author identification is the task of identifying the author of a given text. It can be considered as a typical classification problem, where a set of books with known authors are used for training. The aim is to automatically determine the corresponding author of an anonymous text. 

## Grading = 20 Marks

## Setup Steps

In [ ]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()
  
notebook="U2_MH1_AuthorIdentification" #name of the notebook
Answer = "This notebook is graded by mentors on the day of hackathon"
def setup():
    ipython.magic("sx wget https://cdn.talentsprint.com/talentsprint1/archives/sc/aiml/experiment_related_data/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.rar")
    ipython.magic("sx unrar e /content/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.rar")
    print ("Setup completed successfully")
    return

setup()

Setup completed successfully


### NOTE: You are allowed to use ML libraries such as Sklearn, NLTK etc wherever applicable

### Downloading the required nltk Packages before moving ahead

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from bs4 import BeautifulSoup
import re
import string
import warnings
import numpy as np
import pandas as pd
from collections import Counter
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
nltk.download('wordnet')
warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## **Stage 1:** Dataset Preparation

### 3 Marks -> Ensure you appropriately split the multiple short stories for the below mentioned authors, Which will be your training data.

**1.** Before moving ahead choose two authors based on your team-number allocation: <br/>


Team=1,5,9,13 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    Author-A Vs Author-B <br />
Team=2,6,10,14 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         Author-B Vs Author-C <br />
Team=3,7,11,15 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         Author-C Vs Author-D <br />
Team=4,8,12,16 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;           Author-D Vs Author-E <br />



**2.** Link to the short stories collection of each author for your problem: <br /> 

*   Author-A -> Rudyard Kipling   [Short Stories Collection](http://www.gutenberg.org/files/2781/2781-0.txt) &nbsp;&nbsp;
*   Author-B -> Anton Chekhov [Short Stories Collection](http://www.gutenberg.org/files/1732/1732-0.txt) &nbsp;&nbsp;
*   Author-C -> Guy De Maupassant [Short Stories Collection](http://www.gutenberg.org/cache/epub/21327/pg21327.txt)&nbsp;&nbsp;
*   Author-D -> Mark Twain [Short Stories Collection](http://www.gutenberg.org/files/245/245-0.txt)&nbsp;&nbsp;
*   Author-E -> Saki [Short Stories Collection](http://www.gutenberg.org/files/1477/1477-0.txt)&nbsp;&nbsp;

**Hint for downloading raw text from Gutenberg :**  Refer section "Electronic Books" in the following  [link](https://www.nltk.org/book/ch03.html) for the instructions.  
 


**Hint for finding the index of a text:**   You may use raw.find() and raw.rfind() in the same [link](https://www.nltk.org/book/ch03.html) to find appropriate index of the start and end location 

**Hint for splitting the multiple stories:** Split the stories using long space (white space character)

**Note:** Ignore the table of contents section from the given stories

In [ ]:
# YOUR CODE HERE for downloading and splitting the multiple stories of respective authors which are allocated to you
# Downlaod raw text from Gutenberg corpus
gb = nltk.corpus.gutenberg
print("Gutenberg files:n", gb.fileids()[-5:])

Gutenberg files:n ['milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [ ]:
# Load the Guy story from url and show the length
from urllib import request
url1 = "http://www.gutenberg.org/cache/epub/21327/pg21327.txt"
response1 = request.urlopen(url1)
raw_Guy = response1.read().decode('utf8')
print(len(raw_Guy))
print(type(raw_Guy))

613359
<class 'str'>


In [ ]:
# Load the Mark story from url and show the length
url2 = "http://www.gutenberg.org/files/245/245-0.txt"
response2 = request.urlopen(url2)
raw_Mark = response2.read().decode('utf8')
print(len(raw_Mark))
print(type(raw_Mark))

842802
<class 'str'>


In [ ]:
#@title Default title text
# show a sample of the beginning and ending raw Guy text 
print(raw_Guy[:100])
print(raw_Guy[-100:])

﻿The Project Gutenberg EBook of The Works of Guy de Maupassant, Vol. 1 (of
8), by Guy   de Maupassa
lp produce our new eBooks, and how to
subscribe to our email newsletter to hear about new eBooks.



In [ ]:
#@title Default title text
# show a sample of the beginning and ending raw Mark text 
print(raw_Mark[:100])
print(raw_Mark[-100:])

﻿
The Project Gutenberg EBook of Life On The Mississippi, Complete by Mark
Twain (Samuel Clemens)
roduce our new eBooks, and how to subscribe to
our email newsletter to hear about new eBooks.





In [ ]:
# find the index of first Guy story: The first aim of art, no doubt
print(raw_Guy.find("The first aim of art, no doubt"))
print(raw_Guy.rfind("The first aim of art, no doubt"))


3063
3063


In [ ]:
print(raw_Guy[3063:3093])

The first aim of art, no doubt


In [ ]:
# Ignoring Table of content in Guy story
raw_Guy_no_TC = raw_Guy[3063:]
print(len(raw_Guy_no_TC))
raw_Guy_no_TC[0:299]



610296


'The first aim of art, no doubt, is the representation of things as they\r\nare. But then things are as our eyes see them and as our minds make\r\nthem; and it is thus of primary importance for the critic to distinguish\r\nthe precise qualities of the eyes and minds which make the world into\r\nimaginative '

In [ ]:
# find the index of first Mark story: BUT the basin of the Mississippi is the _Body of The Nation
print(raw_Mark.find("BUT the basin of the Mississippi is the _Body of The Nation"))
print(raw_Mark.rfind("BUT the basin of the Mississippi is the _Body of The Nation"))


12417
12417


In [ ]:
print(raw_Mark[12417:12460]) 

BUT the basin of the Mississippi is the _Bo


In [ ]:
# Ignoring Table of content in Mark story
raw_Mark_no_TC = raw_Mark[12417:]
print(len(raw_Mark_no_TC))

830385


In [ ]:
raw_Guy_no_TC

'The first aim of art, no doubt, is the representation of things as they\r\nare. But then things are as our eyes see them and as our minds make\r\nthem; and it is thus of primary importance for the critic to distinguish\r\nthe precise qualities of the eyes and minds which make the world into\r\nimaginative literature. Reality may be so definite and so false, just as\r\nit may be so fantastic and so true; and, among work which we can\r\napprehend as dealing justly with reality, there may be quite as much\r\ndifference in all that constitutes outward form and likeness as there is\r\nbetween a Dutch interior by Peter van der Hooch, the portrait of a king\r\nby Velasquez, and the image of a woman smiling by Leonardo da Vinci. The\r\nsoul, for instance, is at heart as real as the body; but, as we can hear\r\nit only through the body speaking, and see it only through bodily eyes,\r\nand measure it, often enough, only in the insignificant moment of its\r\naction, it may come to seem to us, at

# By default, split() takes whitespace as the delimiter.



In [ ]:
# split raw Guy to words
word_Guy=raw_Guy_no_TC.split("\r\n\r\n\r\n\r\n\r\n")
print(word_Guy)
print(len(word_Guy))

['The first aim of art, no doubt, is the representation of things as they\r\nare. But then things are as our eyes see them and as our minds make\r\nthem; and it is thus of primary importance for the critic to distinguish\r\nthe precise qualities of the eyes and minds which make the world into\r\nimaginative literature. Reality may be so definite and so false, just as\r\nit may be so fantastic and so true; and, among work which we can\r\napprehend as dealing justly with reality, there may be quite as much\r\ndifference in all that constitutes outward form and likeness as there is\r\nbetween a Dutch interior by Peter van der Hooch, the portrait of a king\r\nby Velasquez, and the image of a woman smiling by Leonardo da Vinci. The\r\nsoul, for instance, is at heart as real as the body; but, as we can hear\r\nit only through the body speaking, and see it only through bodily eyes,\r\nand measure it, often enough, only in the insignificant moment of its\r\naction, it may come to seem to us, a

In [ ]:
# split raw Mark to words
word_Mark = raw_Mark_no_TC.split("\r\n\r\n\r\n\r\n\r\n")
print(word_Mark)
print(len(word_Mark))

["BUT the basin of the Mississippi is the _Body of The Nation_. All the\r\nother parts are but members, important in themselves, yet more important\r\nin their relations to this. Exclusive of the Lake basin and of 300,000\r\nsquare miles in Texas and New Mexico, which in many aspects form a part\r\nof it, this basin contains about 1,250,000 square miles. In extent it is\r\nthe second great valley of the world, being exceeded only by that of the\r\nAmazon. The valley of the frozen Obi approaches it in extent; that of\r\nLa Plata comes next in space, and probably in habitable capacity, having\r\nabout eight-ninths of its area; then comes that of the Yenisei, with\r\nabout seven-ninths; the Lena, Amoor, Hoang-ho, Yang-tse-kiang, and\r\nNile, five-ninths; the Ganges, less than one-half; the Indus, less\r\nthan one-third; the Euphrates, one-fifth; the Rhine, one-fifteenth. It\r\nexceeds in extent the whole of Europe, exclusive of Russia, Norway,\r\nand Sweden. _It would contain austria four

## **Stage 2**: Experiment with Handcrafted features representation
Extract Handcrafted features for the obtained short stories from **Stage-1**

**Stylometry:** 

Each person has a unique vocabulary, sometimes rich, sometimes limited. Although a larger vocabulary is usually associated with literary quality, this is not always the case. Ernest Hemingway is famous for using a surprisingly small number of different words in his writing, which did not prevent him from winning the Nobel Prize for Literature in 1954.

Some people write in short sentences, while others prefer long blocks of text consisting of many clauses. No two people use semicolons, em-dashes, and other forms of punctuation in the same way.




**You may explore the following ways to analyze the text and generate handcrafted features by searching text in a probing way:**

a)  Could the style of punctuation usage help as a handcrafted feature? Both by those who follow punctuations and by those who don't? Interesting [link](https://qwiklit.com/2014/03/05/top-10-authors-who-ignored-the-basic-rules-of-punctuation/) 

b) The same word can sometimes be used in different contexts repeatedly by different authors. Could this fact be converted as a handcrafted feature? [link](https://www.nltk.org/book/ch01.html)

c) The above two are merely examples; As you might have noticed already the NLTK book [link](https://www.nltk.org/book/) offers several methods of analyzing and understanding the text. Each of these analyses is in itself capable of being a handcrafted feature. **However for your evaluation a minimal set of useful handcrafted features which is helping you prove a classification of an is sufficient**

d) Could most command words be used to distinguish authors?  Refer "Counting Vocabulary" section of the [link](https://www.nltk.org/book/ch01.html)

e) How about using a count of most frequently used bi-gram, tri-grams, and using it to classify an author?

f) How about using the frequency histogram of the most frequently used words across the stories by a given author a useful feature? 

The limit here is endlessly limited only by your imagination, and of course your accuracy! :)


### 2 Marks ->  a) List 6 handcrafted features to distinguish author stories.

#As there is no consensus on which stylometric features are the best for authorship identification, We are going to try which of following handcrafted features provid a a good text representation : -
# 1) N-gram frequencies (overlapping n-character frequencies) / Letter frequencies.
# 2) Function word usage (short structure-determining words: common adverbs, auxiliary verbs, conjunctions, determiners, numbers, prepositions and pronouns)
# 3) Vocabulary richness (number of different words used)
# 4) Lexical richness (word frequency as a function of full text)
# 5 ) Distribution of syllables per word Word frequencies
# 6)  Hapax legomena (words used once only)
# 7)  Hapax dislegomena (words used twice only)
# 8)  Word length distribution
# 9) Word collocations (words frequently used together)
# 10 ) Sentence length
# 11) Preferred word positions
# 12) Prepositional phrase structure
# 13)  Distribution parts of speech
# 14) Phrasal composition grammar
# 15) Morphological features – POS tag features. These are the number of nouns; the number of numerals; the number of adjectives; the number of prepositions; the number of verbs; the number of pronouns; the number of interjections; the number of adverbs; the number of рarticles, the number of conjunctions, the number of participles, the number of infinitives, the number of finite verbs.
# 16 ) Unique Words
# 17) First words in a sentence
# 18) Find numbers of different combinations
# 19) None English words

###  4 Marks -> b) Write functions for any 4 of the above 6 handcrafted features and label your authors accordingly.

- Get any 4 hand crafted features from the above listed 6 hand-crafted features for every story obtained from **stage-1**.
- Identify your target variable as author and label them accordingly.

In [ ]:
# Stories_list    UniqueWords    AvgSentLength  punctuation  FrequentWords   Label 
#     1               x1               x2          x3             X4           y 

# YOUR CODE HERE
def Sentences(stories):
    AvgSentLength_Story1 = []
    for i in stories:
        Sent_Story1 = i.split(".")
        average = sum(len(x.split()) for x in Sent_Story1) // len(Sent_Story1)
        AvgSentLength_Story1.append(average)
    return AvgSentLength_Story1


In [ ]:

X1 = Sentences(raw_Guy_no_TC)

In [ ]:
X1

In [ ]:
from nltk import ngrams
def ngrams1(stories):
    unigram = list(ngrams(stories.split(),1))
    return unigram

X2 = ngrams1(raw_Guy_no_TC)

In [ ]:
X2

In [ ]:
ngrams1(raw_Guy_no_TC)

In [ ]:
def bigrams(stories):
    biigram = list(ngrams(stories.split(),2))
    return biigram

X3 = bigrams(raw_Guy_no_TC)

In [ ]:
X3

In [ ]:
bigrams(raw_Mark_no_TC)

In [ ]:
def unique_word(stories):
    return stories.split()

In [ ]:
d={'x1':X1,'x2': X2,'x3':X3, 'x4':X4}
def generate_BiGram(story):
  words=story.split()
  bigram=[]
  for i in range(len(words)-1):
    bigram.append((words[i],words[i+1]))
  return len(bigram)

In [ ]:
from collections import defaultdict
def uniqueWords(stories):
    def_dict = defaultdict(int)
    for word in stories.split():
        if word != ' ':  # exclude spaces
            def_dict[word]+=1
    return def_dict
   
X4 = uniqueWords(raw_Guy_no_TC)

In [ ]:
# pun
import nltk
def punct(stories):
    words = nltk.word_tokenize(stories)
    #words = [word.lower() for word in  words if word.isalpha()]
    words = str.maketrans('', '', string.punctuation)
    return words

In [ ]:
punct(raw_Guy_no_TC)

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [ ]:
import pandas as pd
df = pd.DataFrame(X1,X2)

df

ValueError: ignored

In [ ]:
uniqueWords(raw_Guy_no_TC)

##**Stage 3:** Experiment with Text processing and representation:
Extract features using TFIDF or CountVectorizer or Word2vec for the obtained short stories from **Stage-1**



### 1 Mark -> a) Performing basic cleanup operations such as removing the newline characters and removing trailing spaces

**For example,** Your sentence looks as follows \[' This is a sentence\n\r. Another sentence \n'].

After newline removal from the above example, your sentence will look like \['This is a sentence. Another sentence'].

 In order to do this you can try using a combination of split() and join()

In [ ]:
len(word_Guy), len(word_Mark)

(31, 61)

In [ ]:
# YOUR CODE HERE

stories_Guy = ["".join(word.split("\r\n")) for word in word_Guy]# [word_Guy.split('\n\r')]
stories_Mark = ["".join(word.split("\r\n")) for word in word_Mark]#[word_Mark.split('\n\r')]



In [ ]:
stories_Guy.extend(stories_Mark)
len(stories_Guy)

92

###  5 Marks-> b) Generate vectors for the given stories

Create a representation of text, convert it into vectors (numbers)


**Use any one** of the following algorithms for this task :

* Countvectorizer or
* TFIDFVectorizer or 
* Word2Vec (The word2vec bin file (AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD) can be downloaded as a part of setup  )
  * perform sentence level tokenization and word level tokenization for the given stories

    **Example of sentences as list of words:**<br/>
    **Before:** ['This is a sentence .' , ' Another sentence']<br/>
    **After:** ['This', 'is' ,'a', 'sentence' , ' . ' , ' Another ', ' sentence ' ]

References Documents: 

1.   [Countvectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
2.  [TFIDFVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)


In [ ]:
# YOUR CODE HERE (HINT: Convert to numpy array if needed)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
A = vectorizer.fit_transform(stories_Guy)
X =A.toarray()

In [ ]:
X.shape

(92, 27209)

In [ ]:
stories_Guy_label = np.zeros(len(word_Guy))
stories_Mark_label = np.ones(len(word_Mark))
labels = np.concatenate((stories_Guy_label,stories_Mark_label))

In [ ]:
labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1.])

In [ ]:
labels.shape

(92,)

In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin', binary=True, limit=500000)

In [ ]:
len(stories_Guy)

31

###  1 Mark -> c) Is stop word removal necessary in the context of author identification? Your thoughts below?

In [ ]:
# YOUR ANSWER IN TEXT
#No, It distinishes authors style of writing 

##**Stage 4:** Classification :

### Expected accuracy is above 85%

### 4 Marks -> Perform a classification using either features obtained from Stage2 or Stage3

In [ ]:
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
pred = model.predict(X_test) # this cell takes some time to execute

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.8709677419354839

# Further Ideas for exploration after the hackathon:

**Statistical analysis** of text using NLP, by analysis meaning of sentences, feature based grammars and analyzing structure of sentences! 

reference: www.nltk.org/book